# Tarefa 1a: executar geração de texto

Neste caderno, você aprenderá a usar grandes modelos de linguagem (LLM) para gerar e-mails de resposta destinados a clientes insatisfeitos com a qualidade do atendimento ao cliente prestado pela equipe de suporte. Neste caderno, você gerará um e-mail de agradecimento com base no e-mail anterior do cliente. Você usa o modelo Amazon Titan com a API do Amazon Bedrock e o cliente Boto3.

O prompt usado nesta tarefa é chamado de prompt zero-shot. No prompt zero-shot, você usa uma linguagem simples para descrever a tarefa ou a saída desejada ao modelo de linguagem. O modelo usa os recursos e conhecimento pré-treinado para gerar uma resposta ou concluir a tarefa com base somente no prompt fornecido.

#### Cenário
Você é Bob, um gerente de atendimento ao cliente na UmaEmpresa. Alguns de seus clientes estão insatisfeitos com o atendimento ao cliente e estão reclamando sobre o serviço prestado pela equipe de suporte. Agora, você gostaria de pedir desculpas a esses clientes pelo serviço ruim e recuperar a confiança deles. Você precisa da ajuda de um LLM para gerar vários e-mails de resposta otimistas, personalizados conforme o sentimento descrito no e-mail anterior do cliente.

## Tarefa 1a.1: configuração do ambiente

Nesta tarefa, você configurará o ambiente.

In [2]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1"))

## Tarefa 1a.2: gerar texto

Nesta tarefa, você preparará uma entrada para o serviço Amazon Bedrock gerar um e-mail.

In [3]:
# create the prompt
prompt_data = """
Command:Escreva um e-mail de Bob, Gerente de Atendimento ao Cliente da AnyCompany, para o cliente "John Doe"
que forneceu feedback negativo sobre o serviço prestado pelo nosso engenheiro de suporte ao cliente."""

In [ ]:
# Prepare request body — Amazon Nova Lite format
body = json.dumps({
    "inputText": prompt_data,
    "textGenerationConfig": {
        "maxTokenCount": 1000,
        "temperature": 0.7,
        "topP": 0.9
    }
})

Em seguida, você usará o modelo Amazon Titan.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** o Amazon Titan é compatível com uma janela de contexto de cerca de 4 mil tokens e aceita os seguintes parâmetros:
- `inputText`: prompt para o LLM
- `textGenerationConfig`: esses são os parâmetros que o modelo levará em consideração ao gerar a saída.

A API do Amazon Bedrock fornece uma API `invoke_model` que aceita o seguinte:
- `modelId`: este é o ARN do modelo para os vários modelos de base disponíveis no Amazon Bedrock
- `accept`: o tipo de solicitação de entrada
- `contentType`: o tipo de conteúdo da saída
- `body`: uma string json que consiste no prompt e nas configurações

Consulte a [documentação] (https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para ver os IDs dos modelos de geração de texto disponíveis.

## Tarefa 1a.3: invocar o grande modelo de linguagem Amazon Titan

Nesta tarefa, você explorará como o modelo gera uma saída com base no prompt criado anteriormente.

### Concluir geração de saída

O e-mail é gerado pelo modelo Amazon Titan ao entender a solicitação de entrada e usar seu entendimento inerente de diferentes modalidades. A solicitação à API é síncrona e espera que toda a saída seja gerada pelo modelo.

In [21]:
#invoke model
# Use Amazon Nova Lite como substituto
modelId = 'amazon.nova-lite-v1:0'
accept = 'application/json'
contentType = 'application/json'

try:
    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    # Nova usa formato diferente - verifique a estrutura da resposta
    print("Estrutura da resposta:", response_body.keys())
    
    # Tente extrair o texto (formato pode variar)
    if 'output' in response_body:
        outputText = response_body['output']['message']['content'][0]['text']
    elif 'outputText' in response_body:
        outputText = response_body['outputText']
    else:
        print("Resposta completa:", response_body)
        
    print(outputText)

except Exception as e:
    print(f"Erro: {e}")

Estrutura da resposta: dict_keys(['output', 'stopReason', 'usage'])
Assunto: Resolução do Problema Encontrado e Melhoria do Nosso Serviço

Caro John Doe,

Espero que este e-mail o encontre bem. Olá! Meu nome é Bob e sou o Gerente de Atendimento ao Cliente na AnyCompany. Primeiro, gostaria de lhe agradecer por compartilhar seu feedback conosco. 

Recebemos com seriedade suas preocupações e estamos comprometidos em melhorar nosso serviço. Estamos cientes de que o suporte técnico que você recebeu de nosso engenheiro de suporte ao cliente não atendeu às suas expectativas e, por isso, estamos muito desapontados.

Peço suas desculpas sinceras pelo inconveniente que isso possa ter causado. Entendemos o quão frustrante pode ser lidar com problemas técnicos e esperamos que você entenda que não é a política da AnyCompany oferecer um atendimento insatisfatório.

Nossa equipe está analisando o caso para identificar o que deu errado e tomar as medidas necessárias para garantir que um incidente como

In [48]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)



Caro John Doe,

Espero que este e-mail o encontre bem. Olá! Meu nome é Bob e sou o Gerente de Atendimento ao Cliente na AnyCompany. Primeiro, gostaria de lhe agradecer por compartilhar seu feedback conosco. 

Recebemos com seriedade suas preocupações e estamos comprometidos em melhorar nosso serviço. Estamos cientes de que o suporte técnico que você recebeu de nosso engenheiro de suporte ao cliente não atendeu às suas expectativas e, por isso, estamos muito desapontados.

Peço suas desculpas sinceras pelo inconveniente que isso possa ter causado. Entendemos o quão frustrante pode ser lidar com problemas técnicos e esperamos que você entenda que não é a política da AnyCompany oferecer um atendimento insatisfatório.

Nossa equipe está analisando o caso para identificar o que deu errado e tomar as medidas necessárias para garantir que um incidente como esse não se repita. Além disso, garantimos que o engenheiro de suporte ao cliente que atendeu seu caso será informado sobre suas preocupa

### Transmitir geração de saída

O Bedrock também pode transmitir a saída em forma de blocos, à medida que ela é gerada pelo modelo. Esse e-mail é gerado invocando o modelo com a opção de streaming. `invoke_model_with_response_stream` retorna um `ResponseStream` do qual você pode ler.

In [61]:
# invoke model with response stream
output = []
try:
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
        
    i = 1
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk['bytes'].decode())
            
            # Processa contentBlockDelta (texto real)
            if 'contentBlockDelta' in chunk_obj:
                text = chunk_obj['contentBlockDelta']['delta']['text']
                output.append(text)
                i += 1
                        
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

A abordagem de transmissão com resposta ajuda a obter rapidamente a saída do modelo e permite que o serviço a conclua enquanto você lê. Isso ajuda nos casos de uso em que você solicita que o modelo gere trechos de texto mais longos. Posteriormente, você pode combinar todos os blocos gerados para formar a saída completa e usá-la no seu caso de uso. 

In [63]:
#combine output chunks
print('\n\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


		**COMPLETE OUTPUT**

Assunto: Resolução do Feedback Recebido

Olá John,

Espero que você esteja bem.

Primeiramente, gostaria de agradecer por compartilhar seu feedback sobre a recente experiência que você teve com nosso engenheiro de suporte ao cliente. Lamento profundamente que nossa equipe não tenha atendido às suas expectativas e por qualquer inconveniente que isso possa ter causado.

Na AnyCompany, valorizamos muito a satisfação do cliente e nos esforçamos para fornecer o melhor serviço possível. No entanto, sabemos que ainda há espaço para melhorias e estamos empenhados em garantir que situações como essa não se repitam.

Gostaria de pedir um pouco mais de tempo para investigar minuciosamente o que ocorreu e entender melhor suas preocupações. Além disso, posso providenciar uma reunião com um de nossos engenheiros seniores para discutir suas necessidades e garantir que todas as suas questões sejam devidamente abordadas.

Estamos comprometidos em encontrar uma solução satisfatór


Você testou o uso do SDK boto3, que oferece uma exposição básica à API do Amazon Bedrock. Ao usar essa API, você viu o caso de uso de geração de e-mails de resposta a clientes insatisfeitos.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com **Task1b.ipynb**.